# Big G Express: Predicting Derates
In this project, you will be working with fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. These are indicated by an SPN 5246. 
 
Note that in its raw form the data does not have "labels", so you must define what labels you are going to use and create those labels in your dataset. Also, you will likely need to perform some significant feature engineering in order to build an accurate predictor.

When evaluating the performance of your model, assume that the cost associated with a missed full derate is approximately $4000.00$ in towing and repairs, and the cost of a false positive prediction is about $500 due to having the truck off the road and serviced unnecessarily.

A failed component is usually what triggers this code.

Common Failures

 * Failed DEF doser valve
 * Associated fault code: SPN 5394
 * You ran out of DEF fluid
 * Associated fault code: SPN 5392, SPN 1761
 * Inlet and Outlet NOx sensors failed, not making pressure
 * Associated fault code: SPN 4094
 * EGR system malfunction causing NOX efficiency problems
 * DEF pump failed, not making pressure
 * Associated fault code: SPN 4334, SPN 4339
 * DEF module has failed or DEF harness failure or no power to DEF module causes DEF gauge to be empty and showing datalink error and SCR malfunction.
 * The DEF / ECM could also need updating to eliminate ghost codes.

##### In this notebook, we will:
 * load and compare features between the preprocessed notebooks spns/fmi and the emission control spns/fmi
 * Generate onehotencoded columns for the top faultIDs
 * and other preprocessing that needs to be done, in preparation for fitting the ml model

In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import missingno as msno
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
faults_processed = pd.read_csv('../data/faults_merged_pt1_done.csv', low_memory=False)
faults_processed.head(2)

,Unnamed: 0,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,EngineTimeLtd,FuelLtd,FuelRate,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure,geometry,distance
0,0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,...,1632.2,12300.907429328,0,False,78.8,1023,0,0,POINT (2995931.112894146 658983.4394280221),322511.48
1,5,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,...,9480,70349.809963756,4.583399,True,111.2,1023,13.6022,6.67,POINT (2203912.075923482 -273868.7810324882),1241142.92


#### For some reason, the df reads in with columns that are mixed types. Need to reset some column dtypes.

In [3]:
#For some reason, the df reads in with columns that are mixed types. Need to reset some column dtypes.
# fix data types
faults_processed=faults_processed.drop(columns=['Unnamed: 0'])
faults_processed['EventTimeStamp'] = pd.to_datetime(faults_processed['EventTimeStamp'])
faults_processed['LocationTimeStamp'] = pd.to_datetime(faults_processed['LocationTimeStamp'])

In [4]:
faults_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507342 entries, 0 to 507341
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   RecordID                   507342 non-null  int64         
 1   EventTimeStamp             507342 non-null  datetime64[ns]
 2   spn                        507342 non-null  int64         
 3   fmi                        507342 non-null  int64         
 4   active                     507342 non-null  bool          
 5   activeTransitionCount      507342 non-null  int64         
 6   EquipmentID                507342 non-null  int64         
 7   Latitude                   507342 non-null  float64       
 8   Longitude                  507342 non-null  float64       
 9   LocationTimeStamp          507342 non-null  datetime64[ns]
 10  time_of_day                507342 non-null  object        
 11  BarometricPressure         507342 non-null  object  

In [5]:
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")
sfc.head(2)

C:\Users\micha\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...


In [6]:
sfc=sfc.rename(columns={'Lamp Device':'lamp_device', 
                        'SPN':'spn',
                        'Cummins Description':'cummins_descrip',
                       'Algorithm Description':'algo',
                        'J1939 FMI':'fmi'
                       })

In [7]:

keywords = ['NOx', 'Aftertreatment', 'Emissions', 'DEF', 'Oxidation', 'Catalyst', 'Decomposition', 'SCR', 'DPF']

def filter_function(description):
    return any(re.search(r'\b{}\b'.format(keyword), description, flags=re.IGNORECASE) for keyword in keywords)

sfc_emission_faults = sfc[sfc['cummins_descrip'].apply(filter_function)]

In [8]:
#969 rows.  
sfc_emission_faults.head(2)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,spn,fmi,J2012 Pcode,Lamp Color,lamp_device,cummins_descrip,algo
1061,Y,1663,167,Not Mapped,326,128,13,3241,13,P141A,Amber,Warning,Aftertreatment Exhaust Gas Temperature 1 Swapp...,The aftertreatment diesel oxidation catalyst i...
1062,Y,1664,167,Not Mapped,380,128,11,4796,31,Not Mapped,Amber,Warning,Aftertreatment 1 Diesel Oxidation Catalyst Mis...,The aftertreatment diesel oxidation catalyst i...


In [9]:
sfc_emission_faults[["spn", "fmi"]]=sfc_emission_faults[["spn", "fmi"]].apply(pd.to_numeric)

C:\Users\micha\AppData\Local\Temp\ipykernel_9600\1603515747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sfc_emission_faults[["spn", "fmi"]]=sfc_emission_faults[["spn", "fmi"]].apply(pd.to_numeric)


In [10]:
sfc_emission_faults['spn_combined']=sfc_emission_faults['spn'].astype(str)+'_'+ sfc_emission_faults['fmi'].astype(str)


C:\Users\micha\AppData\Local\Temp\ipykernel_9600\317711213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sfc_emission_faults['spn_combined']=sfc_emission_faults['spn'].astype(str)+'_'+ sfc_emission_faults['fmi'].astype(str)


In [11]:
sfc_emission_faults.info()

<class 'pandas.core.frame.DataFrame'>
Index: 969 entries, 1061 to 5796
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Published in CES 14602  969 non-null    object
 1   Cummins Fault Code      969 non-null    int64 
 2   Revision                969 non-null    int64 
 3   PID                     969 non-null    object
 4   SID                     969 non-null    object
 5   MID                     969 non-null    object
 6   J1587 FMI               969 non-null    int64 
 7   spn                     969 non-null    int64 
 8   fmi                     969 non-null    int64 
 9   J2012 Pcode             969 non-null    object
 10  Lamp Color              836 non-null    object
 11  lamp_device             836 non-null    object
 12  cummins_descrip         969 non-null    object
 13  algo                    190 non-null    object
 14  spn_combined            969 non-null    object
dtypes: int6

In [12]:
sfc_emission_faults.head(2)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,spn,fmi,J2012 Pcode,Lamp Color,lamp_device,cummins_descrip,algo,spn_combined
1061,Y,1663,167,Not Mapped,326,128,13,3241,13,P141A,Amber,Warning,Aftertreatment Exhaust Gas Temperature 1 Swapp...,The aftertreatment diesel oxidation catalyst i...,3241_13
1062,Y,1664,167,Not Mapped,380,128,11,4796,31,Not Mapped,Amber,Warning,Aftertreatment 1 Diesel Oxidation Catalyst Mis...,The aftertreatment diesel oxidation catalyst i...,4796_31


In [13]:

faults_processed['spn_combined']=faults_processed['spn'].astype(str)+'_'+ faults_processed['fmi'].astype(str)

In [14]:
faults_processed.head()

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,FuelLtd,FuelRate,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure,geometry,distance,spn_combined
0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,2015-02-21 11:34:25,...,12300.907429328,0,False,78.8,1023,0,0,POINT (2995931.112894146 658983.4394280221),322511.48,111_17
1,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,2015-02-21 11:40:59,...,70349.809963756,4.583399,True,111.2,1023,13.6022,6.67,POINT (2203912.075923482 -273868.7810324882),1241142.92,111_17
2,7,2015-02-21 11:40:52,111,17,True,2,1597,36.90,-86.44,2015-02-21 11:41:29,...,40961.065436834,14.29175,True,78.8,1023,41.53478,20.59,POINT (2924633.055684726 399381.7435326362),302008.99,111_17
3,11,2015-02-21 11:42:19,111,17,True,1,1582,40.16,-80.15,2015-02-21 11:42:55,...,37724.957799834,13.72378,True,78.8,1023,64.82649,17.69,POINT (3294822.680030754 940816.0324587803),513749.92,111_17
4,15,2015-02-21 11:14:38,1067,2,True,127,309,36.18,-86.70,2015-02-21 11:44:52,...,9487.342989502,0,True,100.4,1279,26.31119,0.58,POINT (2930614.391331102 316197.3132442413),316187.13,1067_2


In [15]:
faults_series=faults_processed.spn_combined
sfc_series=sfc_emission_faults.spn_combined

common_faultcodes=faults_series[faults_series.isin(sfc_series)]

unique_faults=common_faultcodes

In [16]:
unique_faults

32        4364_18
82         3226_4
95        4364_18
121        5848_4
139       3936_15
           ...   
507315     1761_9
507316    1761_19
507317     3031_9
507333    3362_31
507340    3216_10
Name: spn_combined, Length: 20585, dtype: object

In [17]:
one_hot_encoded = pd.get_dummies(unique_faults)

In [18]:
one_hot_encoded

,1761_1,1761_10,1761_11,1761_17,1761_18,1761_19,1761_3,1761_4,1761_9,3031_18,...,5862_3,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4
32,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
82,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
95,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
121,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
139,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507315,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
507316,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
507317,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
507333,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
faults_processed.loc[30:35]

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,FuelLtd,FuelRate,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure,geometry,distance,spn_combined
30,104,2015-02-21 12:55:52,50353,0,True,2,1432,35.89,-84.74,2015-02-21 12:56:28,...,67281.187407724,0,True,46.4,2,52.94277,0.87,POINT (3106399.922361057 348228.4870717321),145026.01,50353_0
31,109,2015-02-21 13:01:31,111,17,True,1,1600,36.48,-84.22,2015-02-21 13:02:08,...,36166.078520982,0,True,53.6,1023,55.48651,2.32,POINT (3126063.34307579 426361.9602302434),98980.72,111_17
32,112,2015-02-21 13:02:17,4364,18,True,1,1442,35.20,-86.69,2015-02-21 12:57:08,...,68055.34360611,14.96539,True,96.8,17407,49.27279,20.3,POINT (2968390.08398245 213428.3950517494),336897.95,4364_18
33,113,2015-02-21 13:04:16,111,17,True,1,1377,36.03,-85.09,2015-02-21 13:04:53,...,73889.58337453,18.21472,True,59,1023,52.93306,26.39,POINT (3071717.927490912 351083.5323678879),173162.18,111_17
34,117,2015-02-21 12:43:18,929,9,True,126,1630,35.80,-86.40,2015-02-21 13:06:20,...,33322.794725306,0.9906483,True,64.4,1279,0,0,POINT (2970299.753093609 285813.7243969291),293501.49,929_9
35,122,2015-02-21 13:09:56,929,9,True,126,1605,36.94,-86.49,2015-02-21 13:10:31,...,32191.609998642,0.4887198,True,86,1279,3.52434,0.29,POINT (2919165.862244155 401619.5793119175),307216.81,929_9


In [20]:
merged_df = pd.merge(faults_processed, one_hot_encoded, how='left', left_index=True, right_index=True)
merged_df

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,5862_3,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4
0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,2015-02-21 11:34:25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,2015-02-21 11:40:59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,2015-02-21 11:40:52,111,17,True,2,1597,36.90,-86.44,2015-02-21 11:41:29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,2015-02-21 11:42:19,111,17,True,1,1582,40.16,-80.15,2015-02-21 11:42:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15,2015-02-21 11:14:38,1067,2,True,127,309,36.18,-86.70,2015-02-21 11:44:52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507337,1248448,2020-03-06 13:12:43,96,3,True,126,1936,30.38,-81.74,2020-03-06 13:29:33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507338,1248452,2020-03-06 13:42:48,111,18,True,93,1886,39.02,-77.03,2020-03-06 13:43:24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507339,1248455,2020-03-06 14:04:23,1569,31,True,5,1994,34.39,-79.46,2020-03-06 14:04:59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507340,1248456,2020-03-06 14:13:38,3216,10,True,1,1850,34.43,-84.92,2020-03-06 14:14:14,...,False,False,False,False,False,False,False,False,False,False


In [21]:
#onehotencoded by hand.  Didn't show in the combined columns...
merged_df['1569_31'] = (merged_df['spn'] == 1569) & (merged_df['fmi'] == 31)

In [22]:
merged_df.fillna(False, inplace=True)

In [23]:
merged_df.head(2)

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4,1569_31
0,1,2015-02-21 10:47:13,111,17,True,2,1439,38.86,-84.63,2015-02-21 11:34:25,...,False,False,False,False,False,False,False,False,False,False
1,6,2015-02-21 11:40:22,111,17,True,1,1417,33.04,-96.18,2015-02-21 11:40:59,...,False,False,False,False,False,False,False,False,False,False


In [24]:
merged_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507342 entries, 0 to 507341
Data columns (total 243 columns):
 #    Column                     Dtype         
---   ------                     -----         
 0    RecordID                   int64         
 1    EventTimeStamp             datetime64[ns]
 2    spn                        int64         
 3    fmi                        int64         
 4    active                     bool          
 5    activeTransitionCount      int64         
 6    EquipmentID                int64         
 7    Latitude                   float64       
 8    Longitude                  float64       
 9    LocationTimeStamp          datetime64[ns]
 10   time_of_day                object        
 11   BarometricPressure         object        
 12   CruiseControlActive        bool          
 13   CruiseControlSetSpeed      float64       
 14   DistanceLtd                object        
 15   EngineCoolantTemperature   object        
 16   EngineLoad        

In [25]:
print(list(merged_df.columns))

['RecordID', 'EventTimeStamp', 'spn', 'fmi', 'active', 'activeTransitionCount', 'EquipmentID', 'Latitude', 'Longitude', 'LocationTimeStamp', 'time_of_day', 'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLtd', 'FuelRate', 'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'Speed', 'TurboBoostPressure', 'geometry', 'distance', 'spn_combined', '1761_1', '1761_10', '1761_11', '1761_17', '1761_18', '1761_19', '1761_3', '1761_4', '1761_9', '3031_18', '3031_2', '3031_3', '3031_4', '3031_9', '3216_10', '3216_11', '3216_16', '3216_2', '3216_20', '3216_21', '3216_3', '3216_4', '3216_9', '3217_2', '3218_2', '3222_5', '3226_10', '3226_11', '3226_16', '3226_2', '3226_20', '3226_21', '3226_4', '3226_9', '3227_10', '3227_21', '3228_2', '3242_0', '3242_15', '3242_16', '3242_3', '3242_4', '3246_0', '3246_15', '3246_16', '3246_2', '3246_3', 

In [26]:
merged_df.to_csv('../data/big_g_pipeline_ready.csv')

faults are likely happening because there are multiple datetime columns. (true, corrected)

Also, strings might be a problem. (need to 

Take the dataset in the log reg

Use the rolling counts of the spn.fmis

group by equipmentId
get a dataframe

In [27]:
merged_df=merged_df.drop('LocationTimeStamp', axis=1)

In [28]:
features = ['1761_1', '1761_10', '1761_11', '1761_17', '1761_18', '1761_19', '1761_3', '1761_4', '1761_9', '3031_18', '3031_2', '3031_3', 
            '3031_4', '3031_9', '3216_10', '3216_11', '3216_16', '3216_2', '3216_20', '3216_21', '3216_3', '3216_4', '3216_9', '3217_2', 
            '3218_2', '3222_5', '3226_10', '3226_11', '3226_16', '3226_2', '3226_20', '3226_21', '3226_4', '3226_9', '3227_10', '3227_21', 
            '3228_2', '3242_0', '3242_15', '3242_16', '3242_3', '3242_4', '3246_0', '3246_15', '3246_16', '3246_2', '3246_3', '3246_4', 
            '3251_0', '3251_10', '3251_15', '3251_16', '3251_2', '3251_3', '3251_4', '3360_11', '3360_12', '3360_19', '3360_2', '3360_9', 
            '3361_12', '3361_2', '3361_3', '3361_4', '3361_5', '3362_31', '3362_7', '3363_16', '3363_3', '3363_4', '3363_5', '3363_7', 
            '3364_10', '3364_11', '3364_18', '3364_3', '3364_9', '3480_17', '3480_2', '3480_3', '3480_4', '3482_2', '3482_3', '3482_7', 
            '3490_3', '3490_4', '3490_7', '3515_10', '3521_18', '3556_18', '3556_2', '3556_5', '3610_2', '3610_3', '3610_4', '3703_31', 
            '3720_15', '3936_14', '3936_15', '3936_16', '3936_7', '4094_18', '4094_31', '4096_31', '4331_16', '4331_18', '4334_16', 
            '4334_18', '4334_2', '4334_3', '4334_4', '4339_7', '4340_3', '4340_4', '4340_5', '4342_3', '4342_4', '4342_5', '4344_3', '4344_4',
            '4344_5', '4346_5', '4360_0', '4360_10', '4360_16', '4360_3', '4360_4', '4363_0', '4363_10', '4363_16', '4363_2', '4363_3', '4363_4',
            '4364_18', '4364_31', '4375_2', '4375_4', '4376_3', '4376_4', '4376_5', '4376_7', '4765_16', '4765_2', '4765_3', '4765_4', '4766_15',
            '4766_3', '4792_14', '4794_31', '4795_31', '4796_31', '5024_10', '5031_10', '520953_4', '521032_14', '5246_16', '5298_17',
            '5298_18', '5319_31', '5392_31', '5394_3', '5394_4', '5394_5', '5394_7', '5397_31', '5491_3', '5491_4', '5491_5', '5491_7', '5569_2',
            '5742_11', '5742_12', '5742_16', '5742_3', '5742_4', '5742_9', '5743_11', '5743_12', '5743_3', '5743_4', '5743_9', '5745_18', '5745_3',
            '5745_4', '5746_4', '5835_21', '5835_3', '5835_4', '5835_9', '5848_12', '5848_13', '5848_19', '5848_4', '5848_9', '5851_18', '5851_2',
            '5853_10', '5862_0', '5862_16', '5862_2', '5862_3', '5862_4', '6773_16', '6780_3', '6802_31', '7321_4', '7323_4', '7854_2', '7854_3',
            '7854_4']

In [29]:
merged_t=merged_df[['EventTimeStamp', 'EquipmentID', '1761_1', '1761_10', '1761_11', '1761_17', '1761_18', '1761_19', '1761_3', '1761_4', '1761_9', '3031_18', '3031_2', '3031_3', 
            '3031_4', '3031_9', '3216_10', '3216_11', '3216_16', '3216_2', '3216_20', '3216_21', '3216_3', '3216_4', '3216_9', '3217_2', 
            '3218_2', '3222_5', '3226_10', '3226_11', '3226_16', '3226_2', '3226_20', '3226_21', '3226_4', '3226_9', '3227_10', '3227_21', 
            '3228_2', '3242_0', '3242_15', '3242_16', '3242_3', '3242_4', '3246_0', '3246_15', '3246_16', '3246_2', '3246_3', '3246_4', 
            '3251_0', '3251_10', '3251_15', '3251_16', '3251_2', '3251_3', '3251_4', '3360_11', '3360_12', '3360_19', '3360_2', '3360_9', 
            '3361_12', '3361_2', '3361_3', '3361_4', '3361_5', '3362_31', '3362_7', '3363_16', '3363_3', '3363_4', '3363_5', '3363_7', 
            '3364_10', '3364_11', '3364_18', '3364_3', '3364_9', '3480_17', '3480_2', '3480_3', '3480_4', '3482_2', '3482_3', '3482_7', 
            '3490_3', '3490_4', '3490_7', '3515_10', '3521_18', '3556_18', '3556_2', '3556_5', '3610_2', '3610_3', '3610_4', '3703_31', 
            '3720_15', '3936_14', '3936_15', '3936_16', '3936_7', '4094_18', '4094_31', '4096_31', '4331_16', '4331_18', '4334_16', 
            '4334_18', '4334_2', '4334_3', '4334_4', '4339_7', '4340_3', '4340_4', '4340_5', '4342_3', '4342_4', '4342_5', '4344_3', '4344_4',
            '4344_5', '4346_5', '4360_0', '4360_10', '4360_16', '4360_3', '4360_4', '4363_0', '4363_10', '4363_16', '4363_2', '4363_3', '4363_4',
            '4364_18', '4364_31', '4375_2', '4375_4', '4376_3', '4376_4', '4376_5', '4376_7', '4765_16', '4765_2', '4765_3', '4765_4', '4766_15',
            '4766_3', '4792_14', '4794_31', '4795_31', '4796_31', '5024_10', '5031_10', '520953_4', '521032_14', '5246_16', '5298_17',
            '5298_18', '5319_31', '5392_31', '5394_3', '5394_4', '5394_5', '5394_7', '5397_31', '5491_3', '5491_4', '5491_5', '5491_7', '5569_2',
            '5742_11', '5742_12', '5742_16', '5742_3', '5742_4', '5742_9', '5743_11', '5743_12', '5743_3', '5743_4', '5743_9', '5745_18', '5745_3',
            '5745_4', '5746_4', '5835_21', '5835_3', '5835_4', '5835_9', '5848_12', '5848_13', '5848_19', '5848_4', '5848_9', '5851_18', '5851_2',
            '5853_10', '5862_0', '5862_16', '5862_2', '5862_3', '5862_4', '6773_16', '6780_3', '6802_31', '7321_4', '7323_4', '7854_2', '7854_3',
            '7854_4']]

In [30]:
merged_t.head()

,EventTimeStamp,EquipmentID,1761_1,1761_10,1761_11,1761_17,1761_18,1761_19,1761_3,1761_4,...,5862_3,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4
0,2015-02-21 10:47:13,1439,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2015-02-21 11:40:22,1417,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2015-02-21 11:40:52,1597,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2015-02-21 11:42:19,1582,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2015-02-21 11:14:38,309,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
merged_t = merged_t.sort_values(by='EventTimeStamp', ascending=False).groupby('EquipmentID')

In [32]:
merged_t=merged_t.rolling('5D', on='EventTimeStamp').sum()

In [33]:
merged_t['5862_3'].sort_values().tail(40)

EquipmentID        
1619         283098    1.0
             283087    1.0
             282946    1.0
             282929    1.0
             282912    1.0
             282860    1.0
             282854    1.0
             282832    1.0
1719         190956    1.0
             190957    1.0
             192220    1.0
             192654    1.0
             192914    1.0
             193031    1.0
             193157    1.0
             193160    1.0
1619         283139    1.0
             283150    1.0
             283168    1.0
             283242    1.0
             283458    1.0
             283440    1.0
             283407    1.0
             283470    1.0
1827         136660    1.0
             136659    1.0
1719         193156    1.0
1619         283360    1.0
             283353    1.0
             283333    1.0
             283323    1.0
             283291    1.0
             283365    1.0
1723         436249    2.0
             436115    2.0
             436094    2.0
        

In [34]:
merged_t.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 507342 entries, (301, 505760) to (108077146, 384172)
Data columns (total 211 columns):
 #    Column          Dtype         
---   ------          -----         
 0    EventTimeStamp  datetime64[ns]
 1    1761_1          float64       
 2    1761_10         float64       
 3    1761_11         float64       
 4    1761_17         float64       
 5    1761_18         float64       
 6    1761_19         float64       
 7    1761_3          float64       
 8    1761_4          float64       
 9    1761_9          float64       
 10   3031_18         float64       
 11   3031_2          float64       
 12   3031_3          float64       
 13   3031_4          float64       
 14   3031_9          float64       
 15   3216_10         float64       
 16   3216_11         float64       
 17   3216_16         float64       
 18   3216_2          float64       
 19   3216_20         float64       
 20   3216_21         float64       
 21   3216_3

In [35]:
merged_t.head(10)

EventTimeStamp  1761_1  1761_10  1761_11  1761_17  \
EquipmentID                                                                 
301         505760 2020-02-20 12:09:01     0.0      0.0      0.0      0.0   
            504526 2020-02-10 13:35:03     0.0      0.0      0.0      0.0   
            504490 2020-02-10 10:44:36     0.0      0.0      0.0      0.0   
            504158 2020-02-07 05:02:13     0.0      0.0      0.0      0.0   
            504120 2020-02-06 17:52:29     0.0      0.0      0.0      0.0   
            502836 2020-01-26 16:48:45     0.0      0.0      0.0      0.0   
            502831 2020-01-26 15:39:12     0.0      0.0      0.0      0.0   
            502823 2020-01-26 14:33:45     0.0      0.0      0.0      0.0   
            502819 2020-01-26 13:48:46     0.0      0.0      0.0      0.0   
            501665 2020-01-17 05:38:33     0.0      0.0      0.0      0.0   

                    1761_18  1761_19  1761_3  1761_4  1761_9  ...  5862_3  \
EquipmentID                                                   ...           
301         505760      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            504526      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            504490      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            504158      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            504120      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            502836      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            502831      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            502823      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            502819      0.0      0.0     0.0     0.0     0.0  ...     0.0   
            501665      0.0      0.0     0.0     0.0     0.0  ...     0.0   

                    5862_4  6773_16  6780_3  6802_31  7321_4  7323_4  7854_2  \
EquipmentID                                                                    
301         505760     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            504526     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            504490     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            504158     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            504120     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            502836     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            502831     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            502823     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            502819     0.0      0.0     0.0      0.0     0.0     0.0     0.0   
            501665     0.0      0.0     0.0      0.0     0.0     0.0     0.0   

                    7854_3  7854_4  
EquipmentID                         
301         505760     0.0     0.0  
            504526     0.0     0.0  
            504490     0.0     0.0  
            504158     0.0     0.0  
            504120     0.0     0.0  
            502836     0.0     0.0  
            502831     0.0     0.0  
            502823     0.0     0.0  
            502819     0.0     0.0  
            501665     0.0     0.0  

[10 rows x 211 columns]

In [36]:
merged_t=merged_t.reset_index().set_index('level_1')


In [37]:
merged_final=pd.merge(merged_df, merged_t, how='inner', left_index=True, right_index=True)

In [38]:
merged_final[['EventTimeStamp_x', 'EventTimeStamp_y']]

,EventTimeStamp_x,EventTimeStamp_y
0,2015-02-21 10:47:13,2015-02-21 10:47:13
1,2015-02-21 11:40:22,2015-02-21 11:40:22
2,2015-02-21 11:40:52,2015-02-21 11:40:52
3,2015-02-21 11:42:19,2015-02-21 11:42:19
4,2015-02-21 11:14:38,2015-02-21 11:14:38
...,...,...
507337,2020-03-06 13:12:43,2020-03-06 13:12:43
507338,2020-03-06 13:42:48,2020-03-06 13:42:48
507339,2020-03-06 14:04:23,2020-03-06 14:04:23
507340,2020-03-06 14:13:38,2020-03-06 14:13:38


In [39]:
print(list(merged_final.columns))

['RecordID', 'EventTimeStamp_x', 'spn', 'fmi', 'active', 'activeTransitionCount', 'EquipmentID_x', 'Latitude', 'Longitude', 'time_of_day', 'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLtd', 'FuelRate', 'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'Speed', 'TurboBoostPressure', 'geometry', 'distance', 'spn_combined', '1761_1_x', '1761_10_x', '1761_11_x', '1761_17_x', '1761_18_x', '1761_19_x', '1761_3_x', '1761_4_x', '1761_9_x', '3031_18_x', '3031_2_x', '3031_3_x', '3031_4_x', '3031_9_x', '3216_10_x', '3216_11_x', '3216_16_x', '3216_2_x', '3216_20_x', '3216_21_x', '3216_3_x', '3216_4_x', '3216_9_x', '3217_2_x', '3218_2_x', '3222_5_x', '3226_10_x', '3226_11_x', '3226_16_x', '3226_2_x', '3226_20_x', '3226_21_x', '3226_4_x', '3226_9_x', '3227_10_x', '3227_21_x', '3228_2_x', '3242_0_x', '3242_15_x', '3242_16_x', '3242_3_x

In [40]:
merged_final.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507342 entries, 0 to 507341
Data columns (total 454 columns):
 #    Column                     Dtype         
---   ------                     -----         
 0    RecordID                   int64         
 1    EventTimeStamp_x           datetime64[ns]
 2    spn                        int64         
 3    fmi                        int64         
 4    active                     bool          
 5    activeTransitionCount      int64         
 6    EquipmentID_x              int64         
 7    Latitude                   float64       
 8    Longitude                  float64       
 9    time_of_day                object        
 10   BarometricPressure         object        
 11   CruiseControlActive        bool          
 12   CruiseControlSetSpeed      float64       
 13   DistanceLtd                object        
 14   EngineCoolantTemperature   object        
 15   EngineLoad                 int64         
 16   EngineOilPressure 

In [41]:
#Columns (13,17,19,20,21,22,23,24,26,28,29) have mixed types

#'EngineCoolantTemperature', 
merged_final['EngineCoolantTemperature']=merged_final['EngineCoolantTemperature'].str.extract('(\d+)')
merged_final['EngineCoolantTemperature']=merged_final['EngineCoolantTemperature'].astype(int)

#'EngineTimeLtd'
merged_final['EngineTimeLtd']=merged_final['EngineTimeLtd'].str.extract('(\d+)')
merged_final['EngineTimeLtd']=merged_final['EngineTimeLtd'].astype(int)

#'EngineRpm'
merged_final['EngineRpm']=merged_final['EngineRpm'].str.extract('(\d+)')
merged_final['EngineRpm']=merged_final['EngineRpm'].astype(int)


In [42]:
#merged_final=merged_final.drop(['EquipmentID_y', 'EventTimeStamp_y', 'RecordID', 'spn', 'fmi', 'active', 'activeTransitionCount', 'EquipmentID_x', 'Latitude', 'Longitude', 'time_of_day', 'rolling_count_5246', 'rolling_count_1569', 'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLtd', 'FuelRate', 'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'Speed', 'TurboBoostPressure', 'geometry', 'distance', 'spn_combined'], axis=1)

In [43]:
merged_final.to_csv('../data/big_g_pipeline_ready.csv')

In [44]:
# Read the saved dataframe
chosen_equipments_dummies = pd.read_csv('../data/big_g_pipeline_ready.csv', index_col= False)

# make the datetime column
chosen_equipments_dummies['EventTimeStamp_x'] = pd.to_datetime(chosen_equipments_dummies['EventTimeStamp_x'])

#sort by datetime
chosen_equipments_dummies.sort_values('EventTimeStamp_x', inplace=True)
#set index
chosen_equipments_dummies.set_index('EventTimeStamp_x', inplace=True)

# function to roll back days after a derate code appears and return the window dataframe
specific_spn = 5246
rollback_days = 7

def check_for_other_spns(group):
    window_data = pd.DataFrame()
    for fault_index in group[group['spn']== specific_spn].index:# when 5246 is encountered  the index is the fault_index
        start_index = fault_index - pd.Timedelta(days=rollback_days) # find the roll back time index
        window_data =pd.concat([window_data, group.loc[start_index:fault_index]]) # create a window dataframe and add it to the empty dataframe
        window_data.loc[window_data.index[-1], 'unique_spn_count'] = window_data['spn'].nunique() # this is a little complicated
        # but since this is inside a loop, i only want to count the most recent one hence .index(-1)
        window_data['multiple_spns'] = window_data['unique_spn_count'] > 1 # boolean based on the code above
    return window_data.reset_index() # return the window dataframe after the loop ends

result = chosen_equipments_dummies.groupby('EquipmentID_x').apply(check_for_other_spns) 

# cleanup result to get a dataframe with eventtimestamp as index
result.drop(columns='index').reset_index(drop = True)
result.set_index('EventTimeStamp_x', inplace=True)
result.drop(columns = "index")

# do the same for the original datafrmae out of caution
chosen_equipment_dummies_reset = chosen_equipments_dummies.reset_index()
chosen_equipment_dummies_reset.set_index('EventTimeStamp_x', inplace=True)

# now we merge on both index and recordid to prevent doublecounting 
final_dataframe = pd.merge(chosen_equipment_dummies_reset, result[['multiple_spns', 'RecordID']], how='left', left_on=['EventTimeStamp_x', 'RecordID'], right_on=['EventTimeStamp_x', 'RecordID'])

# fill all nas with false 
final_dataframe['multiple_spns'].fillna(False, inplace=True)

final_dataframe.to_csv('../data/big_g_pipeline_ready.csv', index=False)

C:\Users\micha\AppData\Local\Temp\ipykernel_9600\998061047.py:2: DtypeWarning: Columns (11,14,17,18,21,22,24,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  chosen_equipments_dummies = pd.read_csv('../data/big_g_pipeline_ready.csv', index_col= False)
